In [ ]:
#DOTA数据集默认png格式图片，暂时把/root/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmrotate/datasets/dota.py里的png全改成bmp

from mmdet.apis import set_random_seed

from mmcv import Config
cfg = Config.fromfile('/opt/data/private/shen/mmrotate-0.3.3/configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90.py')

# Modify dataset type and path


cfg.load_from = '/opt/data/private/shen/mmrotate-0.3.3/checkpoints/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth'


cfg.data.samples_per_gpu=4
cfg.data.workers_per_gpu=4

#classes = ('crab',)     #只有一中目标时，要这么写
'''
cfg.data_root = '/opt/data/private/shen/data/crab/'

cfg.data.test.ann_file = '/opt/data/private/shen/data/crab/test/txt_label_v2/'    #没有测试集标注文件，就和图片路径一样?
cfg.data.test.img_prefix = '/opt/data/private/shen/data/crab/test/images/'
#cfg.data.test.classes = classes

cfg.data.train.ann_file = '/opt/data/private/shen/data/crab/train/txt_label_v2/'   #train和val用的同一个数据集？
cfg.data.train.img_prefix = '/opt/data/private/shen/data/crab/train/images/'
#cfg.data.train.classes = classes

cfg.data.val.ann_file = '/opt/data/private/shen/data/crab/test/txt_label_v2/'
cfg.data.val.img_prefix = '/opt/data/private/shen/data/crab/test/images/'
'''

'''
cfg.data_root = '/opt/data/private/shen/data/DOTA'

cfg.data.test.ann_file = '/opt/data/private/shen/data/DOTA/val/images/'   
cfg.data.test.img_prefix = '/opt/data/private/shen/data/DOTA/val/images/'

cfg.data.train.ann_file = '/opt/data/private/shen/data/DOTA/train/labelTxt-v1.0/'   
cfg.data.train.img_prefix = '/opt/data/private/shen/data/DOTA/train/images/'

cfg.data.val.ann_file = '/opt/data/private/shen/data/DOTA/val/labelTxt-v1.0/'
cfg.data.val.img_prefix = '/opt/data/private/shen/data/DOTA/val/images/'
'''


cfg.data_root = '/opt/data/private/shen/data/split_ss_dota'

cfg.data.test.ann_file = '/opt/data/private/shen/data/split_ss_dota/test/annfiles/'   
cfg.data.test.img_prefix = '/opt/data/private/shen/data/split_ss_dota/test/images/'

cfg.data.train.ann_file = '/opt/data/private/shen/data/split_ss_dota/train/annfiles/'   
cfg.data.train.img_prefix = '/opt/data/private/shen/data/split_ss_dota/train/images/'

cfg.data.val.ann_file = '/opt/data/private/shen/data/split_ss_dota/val/annfiles/'
cfg.data.val.img_prefix = '/opt/data/private/shen/data/split_ss_dota/val/images/'

cfg.work_dir = '/opt/data/private/shen/mmrotate-0.3.3/work_dir/iou'

#cfg.optimizer.lr = 0.005
#cfg.lr_config.warmup = None
cfg.runner.max_epochs = 50
cfg.log_config.interval = 50   #每下降10次展示一次loss

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1  #每迭代1次评估一次mAP
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1  #每迭代3次存储一次训练模型

# Set seed thus the results are more reproducible
cfg.seed = 518
#set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can also use tensorboard to log the training process
#cfg.log_config.hooks = [
#    dict(type='TextLoggerHook'),
#    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
cfg.model.roi_head.bbox_head.loss_bbox=dict(type='RotatedIoULoss')#, loss_weight=1.0)
print(f'Config:\n{cfg.pretty_text}')
cfg.dump(F'{cfg.work_dir}/IoU.py')  #保存配置文件

In [ ]:
#一直爆显存，应该不是transformer算法问题，是数据集路径写错了

import mmcv
import copy
import os.path as osp
import numpy as np

from mmrotate.datasets.builder import DATASETS
from mmrotate.datasets import build_dataset
from mmrotate.models import build_detector
from mmrotate.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

#大数据集载入要挺久